# Model Development
This notebook develops and documents baseline, tree-based, and LSTM models for building cooling load prediction using reusable src modules.

In [10]:
# Setup imports and paths
import os, sys
import numpy as np
import pandas as pd
sys.path.append('..')  # allow importing from src
from src.evaluation.validation import simple_train_test_split
from src.evaluation.metrics import nrmse
from src.models.baseline_models import MeanBaseline, LastValueBaseline, LinearRegressionBaseline
from src.models.tree_models import train_xgboost, train_lightgbm
from src.models.deep_learning import train_lstm
data_path = '../data/processed/features_train.csv'
assert os.path.exists(data_path), f'Missing {data_path}. Run feature engineering first.'

## Load and Prepare Data

In [11]:
df = pd.read_csv(data_path)
target_col = 'Total_Cooling_Load'
# Select numeric features, drop the target
num_df = df.select_dtypes(include=[np.number])
assert target_col in num_df.columns, 'Target column not found in numeric columns'
X = num_df.drop(columns=[target_col]).fillna(num_df.median(numeric_only=True))
y = num_df[target_col].values
X_train, X_test, y_train, y_test = simple_train_test_split(X.values, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7008, 41), (1752, 41), (7008,), (1752,))

## Baseline Models

In [12]:
results = {}
# Mean baseline
mean_model = MeanBaseline().fit(y_train)
y_pred_mean = mean_model.predict(X_test)
results['mean_baseline'] = nrmse(y_test, y_pred_mean)
# Last value baseline
last_model = LastValueBaseline().fit(y_train)
y_pred_last = last_model.predict(X_test)
results['last_value_baseline'] = nrmse(y_test, y_pred_last)
# Linear regression baseline
lin_model = LinearRegressionBaseline().fit(X_train, y_train)
y_pred_lin = lin_model.predict(X_test)
results['linear_regression'] = nrmse(y_test, y_pred_lin)
results

TypeError: got an unexpected keyword argument 'squared'

## Tree-Based Models (XGBoost, LightGBM)

In [ ]:
# These require optional dependencies; gracefully skip if unavailable.
try:
    xgb_model = train_xgboost(X_train, y_train)
    y_pred_xgb = xgb_model.predict(X_test)
    results['xgboost'] = nrmse(y_test, y_pred_xgb)
except Exception as e:
    results['xgboost'] = f'skipped: {e}'

try:
    lgb_model = train_lightgbm(X_train, y_train)
    y_pred_lgb = lgb_model.predict(X_test)
    results['lightgbm'] = nrmse(y_test, y_pred_lgb)
except Exception as e:
    results['lightgbm'] = f'skipped: {e}'

results

## LSTM (Deep Learning)

In [ ]:
# LSTM expects 3D input: (samples, timesteps, features).
# We use a trivial timestep dimension of 1 for demonstration.
try:
    X_train_lstm = np.expand_dims(X_train, axis=1)
    X_test_lstm = np.expand_dims(X_test, axis=1)
    lstm_model = train_lstm(X_train_lstm, y_train)
    y_pred_lstm = lstm_model.predict(X_test_lstm, verbose=0).ravel()
    results['lstm'] = nrmse(y_test, y_pred_lstm)
except Exception as e:
    results['lstm'] = f'skipped: {e}'

results